### 데이터프레임 객체에 함수 매핑
- 데이터프레임 객체를 함수 매핑하려면 pipe() 함수 활용
- 매핑 함수가 반환하는 값에 따라 pipe() 함수가 반환하는 객체의 종류가 결정된다.
- DataFrame객체명.pipe(매핑함수)
- 데이터프레임을 반환하는 경우, 시리즈를 반환하는 경우, 개별 값을 반환하는 경우

In [3]:
# 라이브러리 불러오기
import seaborn as sns
import pandas as pd

# titanic 데이터셋에서 age, fare 2개 열을 선택하여 데이터프레임 만들기

titanic = sns.load_dataset('titanic')
df1 = titanic.loc[:, ['age', 'fare']]
df2 = titanic.loc[:, ['embark_town', 'embarked']]


In [15]:
# 데이터프레임을 전달하면 데이터프레임을 반환
def missing_value(df):
    return df.isna()

# 데이터프레임에 pipe() 함수를 매핑
result_df = df1.pipe(missing_value)
print(result_df.head())
print(type(result_df))

     age   fare
0  False  False
1  False  False
2  False  False
3  False  False
4  False  False
<class 'pandas.core.frame.DataFrame'>


In [19]:
# 데이터프레임을 전달하면 시리즈 객체를 반환
def missing_count(df):
    return missing_value(df).sum()

result_series = df1.pipe(missing_count)
print(result_series)
print(type(result_series))

age     177
fare      0
dtype: int64
<class 'pandas.core.series.Series'>


In [21]:
# 데이터프레임을 전달하면 값을 반환
def total_number_misssing(df):
    return missing_count(df).sum()

result_value = df1.pipe(total_number_misssing)
result_value

177

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   embark_town  889 non-null    object
 1   embarked     889 non-null    object
dtypes: object(2)
memory usage: 14.1+ KB


In [31]:
df2['embark_town'].str[0]

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: embark_town, Length: 891, dtype: object

In [2]:
# chain 함수
def extract_initial(df):
    df['town_initial'] = df['embark_town'].str[0]
    return df

def verify_initial(df):
    df['verified'] = df['embarked'] == df['town_initial']
    return df
# verify_initial(extract_initial(df2))

NameError: name 'df2' is not defined

In [63]:
df2.drop(['town_initial', 'verified'], axis=1, inplace=True)

In [69]:
df2.pipe(extract_initial).pipe(verify_initial) # == verify_initial(extract_initial(df2))

,embark_town,embarked,town_initial,verified
0,Southampton,S,S,True
1,Cherbourg,C,C,True
2,Southampton,S,S,True
3,Southampton,S,S,True
4,Southampton,S,S,True
...,...,...,...,...
886,Southampton,S,S,True
887,Southampton,S,S,True
888,Southampton,S,S,True
889,Cherbourg,C,C,True


### 열 재구성
**열 순서 변경**

In [73]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [83]:
# titanic 데이터셋의 survived 열부터 age열까지 4개의 열을 df로 구성
df = titanic.loc[:, 'survived':'age']
df.head()

,survived,pclass,sex,age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


In [101]:
# 열 이름의 리스트 만들기
type(df.columns.values)
columns = list(df.columns.values)
print(columns)

['survived', 'pclass', 'sex', 'age']


In [111]:
df[['sex', 'age', 'survived', 'pclass']]

,sex,age,survived,pclass
0,male,22.0,0,3
1,female,38.0,1,1
2,female,26.0,1,3
3,female,35.0,1,1
4,male,35.0,0,3
...,...,...,...,...
886,male,27.0,0,2
887,female,19.0,1,1
888,female,NaN,0,3
889,male,26.0,1,1


In [103]:
df[columns]

,survived,pclass,sex,age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0
...,...,...,...,...
886,0,2,male,27.0
887,1,1,female,19.0
888,0,3,female,NaN
889,1,1,male,26.0


In [107]:
# 열 이름을 알파벳 순으로 정렬
col_sorted = sorted(columns, reverse=False)
df_sorted = df[col_sorted]
df_sorted

,age,pclass,sex,survived
0,22.0,3,male,0
1,38.0,1,female,1
2,26.0,3,female,1
3,35.0,1,female,1
4,35.0,3,male,0
...,...,...,...,...
886,27.0,2,male,0
887,19.0,1,female,1
888,NaN,3,female,0
889,26.0,1,male,1


In [113]:
# 열 이름을 내림차순으로 정렬
col_reversed = sorted(columns, reverse=True)
df_reversed = df[col_reversed]
df_reversed

,survived,sex,pclass,age
0,0,male,3,22.0
1,1,female,1,38.0
2,1,female,3,26.0
3,1,female,1,35.0
4,0,male,3,35.0
...,...,...,...,...
886,0,male,2,27.0
887,1,female,1,19.0
888,0,female,3,NaN
889,1,male,1,26.0


In [117]:
# 열 이름을 사용자가 지정한 순서로 정렬
dd = ['sex', 'age', 'survived', 'pclass']
df[dd]

,sex,age,survived,pclass
0,male,22.0,0,3
1,female,38.0,1,1
2,female,26.0,1,3
3,female,35.0,1,1
4,male,35.0,0,3
...,...,...,...,...
886,male,27.0,0,2
887,female,19.0,1,1
888,female,NaN,0,3
889,male,26.0,1,1


## 열 분리

In [120]:
# 주가데이터.xlsx 파일을 불러오기
df = pd.read_excel('주가데이터.xlsx')
df.head()

,연월일,당일종가,전일종가,시가,고가,저가,거래량
0,2018-07-02,10100,600,10850,10900,10000,137977
1,2018-06-29,10700,300,10550,10900,9990,170253
2,2018-06-28,10400,500,10900,10950,10150,155769
3,2018-06-27,10900,100,10800,11050,10500,133548
4,2018-06-26,10800,350,10900,11000,10700,63039


In [122]:
# 연월일 => 연, 월, 일 3개의 컬럼으로 분리  => 데이터타입 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   연월일     20 non-null     datetime64[ns]
 1   당일종가    20 non-null     int64         
 2   전일종가    20 non-null     int64         
 3   시가      20 non-null     int64         
 4   고가      20 non-null     int64         
 5   저가      20 non-null     int64         
 6   거래량     20 non-null     int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 1.2 KB


In [128]:
# 연월일 => datetime64로 되어있어서 열 분리하기 위해서 데이터타입 변경 => astype() 함수 이용 'str'로 변경
df['연월일'] =  df['연월일'].astype('str')
dates = df['연월일'].str.split('-')

In [136]:
dates

0     [2018, 07, 02]
1     [2018, 06, 29]
2     [2018, 06, 28]
3     [2018, 06, 27]
4     [2018, 06, 26]
5     [2018, 06, 25]
6     [2018, 06, 22]
7     [2018, 06, 21]
8     [2018, 06, 20]
9     [2018, 06, 19]
10    [2018, 06, 18]
11    [2018, 06, 15]
12    [2018, 06, 14]
13    [2018, 06, 12]
14    [2018, 06, 11]
15    [2018, 06, 08]
16    [2018, 06, 07]
17    [2018, 06, 05]
18    [2018, 06, 04]
19    [2018, 06, 01]
Name: 연월일, dtype: object

In [138]:
dates.str[0].head()

0    2018
1    2018
2    2018
3    2018
4    2018
Name: 연월일, dtype: object

In [148]:
df['연'] = dates.str.get(0)
df['월'] = dates.str[1]
df['일'] = dates.str[2]

In [150]:
df.head()

,연월일,당일종가,전일종가,시가,고가,저가,거래량,연,월,일
0,2018-07-02,10100,600,10850,10900,10000,137977,2018,07,02
1,2018-06-29,10700,300,10550,10900,9990,170253,2018,06,29
2,2018-06-28,10400,500,10900,10950,10150,155769,2018,06,28
3,2018-06-27,10900,100,10800,11050,10500,133548,2018,06,27
4,2018-06-26,10800,350,10900,11000,10700,63039,2018,06,26


In [154]:
print(df['연월일'].dtypes)

object


In [156]:
df['연월일'].head()

0    2018-07-02
1    2018-06-29
2    2018-06-28
3    2018-06-27
4    2018-06-26
Name: 연월일, dtype: object

In [164]:
df_expand = df['연월일'].str.split('-', expand=True)
# print(df_expand)
df_expand.columns = ['연', '월', '일']
print(df_expand)
print(type(df_expand))

       연   월   일
0   2018  07  02
1   2018  06  29
2   2018  06  28
3   2018  06  27
4   2018  06  26
5   2018  06  25
6   2018  06  22
7   2018  06  21
8   2018  06  20
9   2018  06  19
10  2018  06  18
11  2018  06  15
12  2018  06  14
13  2018  06  12
14  2018  06  11
15  2018  06  08
16  2018  06  07
17  2018  06  05
18  2018  06  04
19  2018  06  01
<class 'pandas.core.frame.DataFrame'>


### 필터링
**불린 인덱싱**
- 시리즈 또는 데이터프레임의 데이터 중에서 특정 조건식을 만족하는 원소만 따로 추출하는 개념
- 시리즈 객체에 어떤 조건식을 적용하면 각 원소에 대해 참/거짓을 판별하여 불린(참, 거짓) 값으로 구성된 시리즈를 반환
- 참에 해당하는 데이터 값을 따로 선택할 수 있는데 많은 데이터 중에서 어떤 조건을 만족하는 데이터만을 추출하는 필터링 기법
- 데이터프레임의 각 열은 시리즈 객체이므로, 조건식(>, < , ==, ...)을 적용하면 각 원소가 조건을 만족하는지 여부를 참과 거짓 값으로 표시하여 불린 시리즈를 만들 수 있다. 이 불린 시리즈를 데이터프레임에 대입하면 조건을 만족하는 행들만 선택할 수 있다.

In [183]:
# titanic 데이터셋에서 'age' 열의 값이 10보다 크거나 같고 20보다 작은 두 조건식을 and(&)로 결합한 조건 작성
# titanic.info()
mask1 = ( titanic.age >= 10 ) & ( titanic.age < 20  )
mask1

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887     True
888    False
889    False
890    False
Name: age, Length: 891, dtype: bool

In [189]:
# 위에서 저장한 불린 시리즈 배열을 데이터프레임의 행 인덱스 위치에 대입하면 해당 조건을 만족하는 행만 남게 된다.
df_10 = titanic.loc[mask1 , ['survived', 'sex', 'age', 'fare' ]]
df_10.head()

,survived,sex,age,fare
9,1,female,14.0,30.0708
14,0,female,14.0,7.8542
22,1,female,15.0,8.0292
27,0,male,19.0,263.0000
38,0,female,18.0,18.0000


In [195]:
# 서로 다른 두 열('age', 'sex')에 조건식을 적용해 보자  => 불린 시리즈
# 나이가 10세 미만이고, 성별이 여성인 객체들만 따로 선택
mask2 = ( titanic.age < 10 ) & ( titanic.sex == 'female' )
con_df = titanic.loc[mask2 , : ]
con_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
24,0,3,female,8.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
43,1,2,female,3.0,1,2,41.5792,C,Second,child,False,NaN,Cherbourg,yes,False
58,1,2,female,5.0,1,2,27.7500,S,Second,child,False,NaN,Southampton,yes,False
119,0,3,female,2.0,4,2,31.2750,S,Third,child,False,NaN,Southampton,no,False


In [199]:
# 두 조건식을 or(|) 연산자로 결합하여 두 조건 중 하나라도 참인 값을 추출
# 나이가 10세 미만 또는 60세 이상 데이터 필터링
mask3 = ( titanic.age < 10 ) | ( titanic.age >= 60 )
con2_df = titanic.loc[mask3, :]
con2_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
7,0,3,male,2.0,3,1,21.075,S,Third,child,False,NaN,Southampton,no,False
10,1,3,female,4.0,1,1,16.700,S,Third,child,False,G,Southampton,yes,False
16,0,3,male,2.0,4,1,29.125,Q,Third,child,False,NaN,Queenstown,no,False
24,0,3,female,8.0,3,1,21.075,S,Third,child,False,NaN,Southampton,no,False
33,0,2,male,66.0,0,0,10.500,S,Second,man,True,NaN,Southampton,no,True


**데이터프레임을 불러와서 열의 순서를 재배열할 수 있고, 조건을 지정해서 조건에 만족하는 행의 데이터만 추출할 수 있다.**

**isin() 함수 활용**
- 데이터프레임에서 특정 열에 isin() 함수를 적용하면 True/False을 구할 수 있고, 이 불린 시리즈를 통해서 조건에 만족하는 행들을 추출할 수 있다.

In [205]:
# 'sibsp' 열의 값이 3, 4, 5 중에서 하나인 행들을 추출
mask4 = ( titanic.sibsp == 3 ) | ( titanic.sibsp == 4 ) | ( titanic.sibsp == 5)
sib_df = titanic.loc[mask4, : ]
sib_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
16,0,3,male,2.0,4,1,29.1250,Q,Third,child,False,NaN,Queenstown,no,False
24,0,3,female,8.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
27,0,1,male,19.0,3,2,263.0000,S,First,man,True,C,Southampton,no,False
50,0,3,male,7.0,4,1,39.6875,S,Third,child,False,NaN,Southampton,no,False


In [209]:
isin_filter = titanic['sibsp'].isin([3, 4, 5])
titanic.loc[isin_filter, :].head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
16,0,3,male,2.0,4,1,29.1250,Q,Third,child,False,NaN,Queenstown,no,False
24,0,3,female,8.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
27,0,1,male,19.0,3,2,263.0000,S,First,man,True,C,Southampton,no,False
50,0,3,male,7.0,4,1,39.6875,S,Third,child,False,NaN,Southampton,no,False


### 데이터프레임 합치기
- 데이터프레임을 하나로 합치거나 연결할 때는 concat(), merge(), join() 등 함수를 이용

**데이터프레임 연결**
- concat() 함수를 이용
- 연결하는 방향 : 행(axis=0 / 위아래로 연결), 열(axis=1 / 좌우로 연결)

In [219]:
df1 = pd.DataFrame({'a':['a0', 'a1', 'a2','a3'],
                    'b':['b0', 'b1', 'b2','b3'],
                    'c':['c0', 'c1', 'c2','c3']}, index = [0, 1, 2, 3])

df2 = pd.DataFrame({'a':['a2', 'a3', 'a4','a5'],
                    'b':['b2', 'b3', 'b4','b5'],
                    'c':['c2', 'c3', 'c4','c5'],
                   'd':['d2', 'd3', 'd4','d5']}, index = [2, 3, 4, 5])
print(df1)
print(df2)

    a   b   c
0  a0  b0  c0
1  a1  b1  c1
2  a2  b2  c2
3  a3  b3  c3
    a   b   c   d
2  a2  b2  c2  d2
3  a3  b3  c3  d3
4  a4  b4  c4  d4
5  a5  b5  c5  d5


In [221]:
result1 = pd.concat([df1, df2])
print(result1)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
2  a2  b2  c2   d2
3  a3  b3  c3   d3
4  a4  b4  c4   d4
5  a5  b5  c5   d5


In [225]:
result2 = pd.concat([df1, df2], axis= 0)
print(result2)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
2  a2  b2  c2   d2
3  a3  b3  c3   d3
4  a4  b4  c4   d4
5  a5  b5  c5   d5


In [227]:
result3 = pd.concat([df1, df2], axis= 1)
print(result3)

     a    b    c    a    b    c    d
0   a0   b0   c0  NaN  NaN  NaN  NaN
1   a1   b1   c1  NaN  NaN  NaN  NaN
2   a2   b2   c2   a2   b2   c2   d2
3   a3   b3   c3   a3   b3   c3   d3
4  NaN  NaN  NaN   a4   b4   c4   d4
5  NaN  NaN  NaN   a5   b5   c5   d5


In [231]:
result4 = pd.concat([df1, df2], ignore_index=True)
print(result4)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
4  a2  b2  c2   d2
5  a3  b3  c3   d3
6  a4  b4  c4   d4
7  a5  b5  c5   d5


**데이터프레임 병합**
- merge() 함수를 이용
- 두 개의 데이터프레임을 합쳐서 하나의 데이터프레임으로 만들어 준다. 기준이 되는 열이나 인덱스를 키(key)라고 한다.
- 키가 되는 열이나 인덱스는 반드시 양쪽 데이터프레임에 모두 존재해야한다.

In [235]:
# stock_price.xlsx, stock_valuation.xlsx 파일을 불러오기
df1 = pd.read_excel('stock_price.xlsx')
df2 = pd.read_excel('stock_valuation.xlsx')
print(df1)
print(df2)

       id stock_name          value   price
0  128940       한미약품   59385.666667  421000
1  130960     CJ E&M   58540.666667   98900
2  138250      엔에스쇼핑   14558.666667   13200
3  139480        이마트  239230.833333  254500
4  142280     녹십자엠에스     468.833333   10200
5  145990        삼양사   82750.000000   82000
6  185750        종근당   40293.666667  100500
7  192400      쿠쿠홀딩스  179204.666667  177500
8  199800         툴젠   -2514.333333  115400
9  204210     모두투어리츠    3093.333333    3475
       id       name           eps     bps        per       pbr
0  130960     CJ E&M   6301.333333   54068  15.695091  1.829178
1  136480         하림    274.166667    3551  11.489362  0.887074
2  138040    메리츠금융지주   2122.333333   14894   6.313806  0.899691
3  139480        이마트  18268.166667  295780  13.931338  0.860437
4  145990        삼양사   5741.000000  108090  14.283226  0.758627
5  161390      한국타이어   5648.500000   51341   7.453306  0.820007
6  181710  NHN엔터테인먼트   2110.166667   78434  30.755864  0.827447
7  1

In [237]:
# merge() 함수는 기본 옵션(on=None 으로 지정하면 두 데이터프레임에 공통으로 속하는 모든 열을 기준으로 병합, 디폴트)
# how = 'inner' : 병합시 기준이 되는 열의 데이터가 양쪽에 모두 존재할 때 결과로 구함, 디폴트
# how = 'outer' : 양쪽에서 한쪽에만 존재해도 결과로 구함
merge_basic = pd.merge(df1, df2 )
print(merge_basic)

       id stock_name          value   price    name           eps     bps  \
0  130960     CJ E&M   58540.666667   98900  CJ E&M   6301.333333   54068   
1  139480        이마트  239230.833333  254500     이마트  18268.166667  295780   
2  145990        삼양사   82750.000000   82000     삼양사   5741.000000  108090   
3  185750        종근당   40293.666667  100500     종근당   3990.333333   40684   
4  204210     모두투어리츠    3093.333333    3475  모두투어리츠     85.166667    5335   

         per       pbr  
0  15.695091  1.829178  
1  13.931338  0.860437  
2  14.283226  0.758627  
3  25.185866  2.470259  
4  40.802348  0.651359  


In [239]:
merge_on_how = pd.merge(df1, df2 ,on=None, how='inner')
print(merge_on_how)

       id stock_name          value   price    name           eps     bps  \
0  130960     CJ E&M   58540.666667   98900  CJ E&M   6301.333333   54068   
1  139480        이마트  239230.833333  254500     이마트  18268.166667  295780   
2  145990        삼양사   82750.000000   82000     삼양사   5741.000000  108090   
3  185750        종근당   40293.666667  100500     종근당   3990.333333   40684   
4  204210     모두투어리츠    3093.333333    3475  모두투어리츠     85.166667    5335   

         per       pbr  
0  15.695091  1.829178  
1  13.931338  0.860437  
2  14.283226  0.758627  
3  25.185866  2.470259  
4  40.802348  0.651359  


In [241]:
merge_onid_how = pd.merge(df1, df2 ,on='id', how='inner')
print(merge_onid_how)

       id stock_name          value   price    name           eps     bps  \
0  130960     CJ E&M   58540.666667   98900  CJ E&M   6301.333333   54068   
1  139480        이마트  239230.833333  254500     이마트  18268.166667  295780   
2  145990        삼양사   82750.000000   82000     삼양사   5741.000000  108090   
3  185750        종근당   40293.666667  100500     종근당   3990.333333   40684   
4  204210     모두투어리츠    3093.333333    3475  모두투어리츠     85.166667    5335   

         per       pbr  
0  15.695091  1.829178  
1  13.931338  0.860437  
2  14.283226  0.758627  
3  25.185866  2.470259  
4  40.802348  0.651359  


In [243]:
merge_onid_howouter = pd.merge(df1, df2 ,on='id', how='outer')
print(merge_onid_howouter)

        id stock_name          value     price       name           eps  \
0   128940       한미약품   59385.666667  421000.0        NaN           NaN   
1   130960     CJ E&M   58540.666667   98900.0     CJ E&M   6301.333333   
2   136480        NaN            NaN       NaN         하림    274.166667   
3   138040        NaN            NaN       NaN    메리츠금융지주   2122.333333   
4   138250      엔에스쇼핑   14558.666667   13200.0        NaN           NaN   
5   139480        이마트  239230.833333  254500.0        이마트  18268.166667   
6   142280     녹십자엠에스     468.833333   10200.0        NaN           NaN   
7   145990        삼양사   82750.000000   82000.0        삼양사   5741.000000   
8   161390        NaN            NaN       NaN      한국타이어   5648.500000   
9   181710        NaN            NaN       NaN  NHN엔터테인먼트   2110.166667   
10  185750        종근당   40293.666667  100500.0        종근당   3990.333333   
11  192400      쿠쿠홀딩스  179204.666667  177500.0        NaN           NaN   
12  199800         툴젠   -

In [247]:
merge_onleftrightid = pd.merge(df1, df2 ,left_on='stock_name', right_on='name', how='inner')
print(merge_onleftrightid)

     id_x stock_name          value   price    id_y    name           eps  \
0  130960     CJ E&M   58540.666667   98900  130960  CJ E&M   6301.333333   
1  139480        이마트  239230.833333  254500  139480     이마트  18268.166667   
2  145990        삼양사   82750.000000   82000  145990     삼양사   5741.000000   
3  185750        종근당   40293.666667  100500  185750     종근당   3990.333333   
4  204210     모두투어리츠    3093.333333    3475  204210  모두투어리츠     85.166667   

      bps        per       pbr  
0   54068  15.695091  1.829178  
1  295780  13.931338  0.860437  
2  108090  14.283226  0.758627  
3   40684  25.185866  2.470259  
4    5335  40.802348  0.651359  


In [249]:
merge_onleftrightid = pd.merge(df1, df2 ,left_on='stock_name', right_on='name')
print(merge_onleftrightid)

     id_x stock_name          value   price    id_y    name           eps  \
0  130960     CJ E&M   58540.666667   98900  130960  CJ E&M   6301.333333   
1  139480        이마트  239230.833333  254500  139480     이마트  18268.166667   
2  145990        삼양사   82750.000000   82000  145990     삼양사   5741.000000   
3  185750        종근당   40293.666667  100500  185750     종근당   3990.333333   
4  204210     모두투어리츠    3093.333333    3475  204210  모두투어리츠     85.166667   

      bps        per       pbr  
0   54068  15.695091  1.829178  
1  295780  13.931338  0.860437  
2  108090  14.283226  0.758627  
3   40684  25.185866  2.470259  
4    5335  40.802348  0.651359  


In [251]:
merge_onleftrightid = pd.merge(df1, df2 ,left_on='stock_name', right_on='name', how='left')
print(merge_onleftrightid)

     id_x stock_name          value   price      id_y    name           eps  \
0  128940       한미약품   59385.666667  421000       NaN     NaN           NaN   
1  130960     CJ E&M   58540.666667   98900  130960.0  CJ E&M   6301.333333   
2  138250      엔에스쇼핑   14558.666667   13200       NaN     NaN           NaN   
3  139480        이마트  239230.833333  254500  139480.0     이마트  18268.166667   
4  142280     녹십자엠에스     468.833333   10200       NaN     NaN           NaN   
5  145990        삼양사   82750.000000   82000  145990.0     삼양사   5741.000000   
6  185750        종근당   40293.666667  100500  185750.0     종근당   3990.333333   
7  192400      쿠쿠홀딩스  179204.666667  177500       NaN     NaN           NaN   
8  199800         툴젠   -2514.333333  115400       NaN     NaN           NaN   
9  204210     모두투어리츠    3093.333333    3475  204210.0  모두투어리츠     85.166667   

        bps        per       pbr  
0       NaN        NaN       NaN  
1   54068.0  15.695091  1.829178  
2       NaN        NaN   

**데이터프레임 결합**
- join() 함수를 이용, 기본 작동 방식은 merge()와 비슷하다.
- join() 함수는 두 데이터프레임의 행 인덱스를 기준으로 결합